In [ ]:
import ee
import geemap
from gee_0_utils import *

initialize()
data_folder, roi = global_variables()

## Surrounding mature forests

- Mean biomass of surrounding mature forests (interpolated with Gaussian kernel)
- Mean fragmentation of surrounding mature forests (interpolated with Gaussian kernel)
- Total surrounding mature forest cover (normalized from 0-1)

In [ ]:
mature_mask = lulc_all.eq(3).reduce(ee.Reducer.allNonZero()).selfMask()
edge_detec = mature_mask.unmask(-1).zeroCrossing()
distance_to_edge = edge_detec.fastDistanceTransform().sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).rename("distance_to_edge")
distance_to_edge = distance_to_edge.gt(1000).selfMask()
distance_to_edge = distance_to_edge.updateMask(mature_mask)
mature_biomass = biomass_all.updateMask(distance_to_edge).rename("mature_biomass")

sur_cover = mature_mask.unmask(0).focalMean(radius=1000, units="meters").rename("sur_cover")

if export_mature_forests:

    task = ee.batch.Export.image.toAsset(
        image=mature_biomass,
        description="mature_biomass",
        assetId="projects/amazon-forest-regrowth/assets/mature_biomass",
        region=age.geometry(),
        crs="EPSG:4326",
        crsTransform=proj['transform'],
        maxPixels=4e12
    )
    # Start the export task
    task.start()

    export_image(sur_cover, "mature_biomass_500m", scale = 500)

    export_image(sur_cover, "sur_cover", scale = 30)

In [ ]:
gaus_kernel = ee.Kernel.gaussian(radius=1000, sigma=200, units="meters")
mat_gaus_ker = (
    mature_biomass.focalMean(
        radius = 1000, units = "meters", kernel = gaus_kernel, iterations = 1
    )
    .rename("mat_biomass")
    .float()
)


# the fragmentation data also does not cover the entire landscape - it is targeted towards mature forests.
# frag file from Ma et al was originally reprojected in QGIS into WGS84 (original projection is not compatible with google earth engine)
# frag values vary from 0 to 1
frag = ee.Image(f"{data_folder}/raw/frag").rename("frag").clip(age.geometry())
frag_ker = frag.focalMean(radius=5000, units="meters").rename("frag")



if export_mature_forests:
    export_image(mat_gaus_ker, "mat_gaus_ker", scale=30)
    export_image(frag_ker, "frag_ker", scale=30)
